In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from detection.visualization import show_loss_history

%load_ext autoreload
%autoreload 2

# Analyze results

## Read results DataFrame and compute position error

In [ ]:
root = 'checkpoints'

results_folders = ['tracknet_v2_rnn_360_640/phase_2_0/checkpoint_0005_results/',
                   'tracknet_v2_rnn_360_640/phase_2_1/checkpoint_0005_results/',
                   'tracknet_v2_rnn_360_640/phase_3_0/checkpoint_0002_results/',
                   'tracknet_v2_mse_360_640/checkpoint_0027_results/',
                   'tracknet_v2_mse_360_640_4f/checkpoint_0020_results/',
                   'tracknet_v2_mse_360_640_2f/checkpoint_0019_results/',
                   ]

errors = [] # list of square errors for each
max_values = []
image_size = (360, 640)

for folder in results_folders:
    df_out = pd.read_csv(os.path.join(root, folder, 'output_val.csv'))
    max_value = df_out['max_values'].values
    max_values.append(max_value)

    x_true, x_pred, y_true, y_pred = [df_out[k].values for k in ['x_true', 'x_pred', 'y_true', 'y_pred']]
    error = (np.sqrt((image_size[1] * (x_true-x_pred))**2 + (image_size[0] * (y_true-y_pred))**2))
    errors.append(error)

In [ ]:
df = pd.DataFrame({'max_value': np.concatenate(max_values), 'error': np.concatenate(errors), 'model': [str(i) for i in range(len(errors)) for _ in range(len(errors[0]))]})

In [ ]:
import seaborn as sns

i = -2
# sns.kdeplot(x=max_values[i], y=errors[i], fill=True, cmap='mako', levels=100, thresh=0, clip=((0,1), (0,15)))
sns.histplot(x=max_values[i], y=errors[i], bins=(np.linspace(0, 1, 50), np.arange(15)))

plt.show()

## Analyze error

In [ ]:
thresholds = [1, 5, 10]

summary = []
for folder, error in zip(results_folders, errors):
    result = {'folder': folder,
              'mean': np.mean(error),
              'std': np.std(error),
              'median': np.median(error)}

    hist, bins = np.histogram(error, bins=np.arange(np.max(error)), density=True)
    for t in thresholds:
        result[f'{t} px'] = hist[:t].sum()

    summary.append(result)

pd.DataFrame(summary)

In [ ]:
def print_summary(summary: dict):
    for k, v in summary.items():
        print_v = v
        if 'px' in k:
            print_v = f'{100*v:.2g}%'
        elif k!='folder':
            print_v = f'{v:.2g}'
        print(f"{k}: {print_v}")

hist_range = thresholds[-1]
for s, error in zip(summary, errors):
    print_summary(s)

    plt.hist(error.clip(max=hist_range), bins=np.arange(hist_range+1), density=True, align='left', rwidth=0.8)
    plt.ylim(0, 1)
    plt.show()